In [1]:
#### label 정보
#### (배경 - 0 / 몸통 - 1 / 오른손 - 2 / 왼손 - 3 / 왼발 - 4 / 오른발 - 5 / 오른쪽 허벅지 - 6 / 왼쪽 허벅지 - 7 / 오른쪽 종아리 - 8 / 왼쪽 종아리 - 9 / 왼쪽 팔 - 10 / 
#### 오른쪽 팔 - 11 / 왼쪽 전완 - 12 / 오른쪽 전완 - 13 / 머리 - 14)
#### 해당 값이 RGB 값임 ex) 배경 0,0,0
### mask 확장자 png input image 확장자 jpg 해깔리지 말것

In [2]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)
import mmcv
import os.path as osp

1.9.0 True
0.18.0


In [3]:
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette

In [4]:
classes = ('bg', 'body', 'right_hand', 'left_hand', 'left_leg', 'right_reg', 'right_thigh', 'left_thigh','right_calf','left_calf'
           ,'left_arm','right_arm','left_forearm','right_forearm','head')

In [5]:
palette = [[0,0,0], [1,1,1], [2,2,2], [3,3,3],[4,4,4], [5,5,5], [6,6,6], [7,7,7], [8,8,8], [9,9,9], [10,10,10], [11,11,11], [12,12,12], [13,13,13], [14,14,14]]

In [6]:
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset

@DATASETS.register_module()
class body_seg(CustomDataset):
  CLASSES = classes
  PALETTE = palette
  def __init__(self, split, **kwargs):
    super().__init__(img_suffix='.jpg', seg_map_suffix='.png', 
                     split=split, **kwargs)
    assert osp.exists(self.img_dir) and self.split is not None


In [7]:
from mmcv import Config

In [8]:
from mmseg.apis import set_random_seed
cfg = Config.fromfile('mmsegmentation/configs/swin/upernet_swin_small_patch4_window7_512x512_160k_ade20k_pretrain_224x224_1K.py')

In [9]:
cfg.dataset_type = 'body_seg'
cfg.data_root = '/root'


# Since we use ony one GPU, BN is used instead of SyncBN
## encoding 부분은 layer normalization 
## encoding 이후 ecoder part 부터는 batchnormalization
cfg.norm_cfg = dict(type='LN', requires_grad=True)
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.auxiliary_head.norm_cfg = dict(type='BN', requires_grad=True)


# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 15
cfg.model.auxiliary_head.num_classes = 15

cfg.data.samples_per_gpu = 24
cfg.data.workers_per_gpu = 2

cfg.img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
cfg.crop_size = (224, 224)

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(256, 256), ratio_range=(0.5, 2.0)),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='CLAHE'),
    dict(type='Normalize', **cfg.img_norm_cfg),
    dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='CLAHE'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(224, 224),
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='Normalize', **cfg.img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]

cfg.data.train.type = cfg.dataset_type
cfg.data.train.data_root = cfg.data_root
cfg.data.train.img_dir = 'train2014'
cfg.data.train.ann_dir = 'train_mask'
cfg.data.train.pipeline = cfg.train_pipeline
cfg.data.train.split = 'splits/train.txt'

cfg.data.val.type = cfg.dataset_type
cfg.data.val.data_root = cfg.data_root
cfg.data.val.img_dir = 'val2014'
cfg.data.val.ann_dir = 'val_mask'
cfg.data.val.pipeline = cfg.test_pipeline
cfg.data.val.split = 'splits/val.txt'


cfg.data.test.type = cfg.dataset_type
cfg.data.test.data_root = cfg.data_root
cfg.data.test.img_dir = 'val2014'
cfg.data.test.ann_dir = 'val_mask'
cfg.data.test.pipeline = cfg.test_pipeline
cfg.data.test.split = 'splits/val.txt'

checkpoint_file = 'pretrained/upernet_swin_small_patch4_window7_512x512_160k_ade20k_pretrain_224x224_1K.pth'
cfg.load_from = checkpoint_file

# Set up working dir to save files and logs.
cfg.work_dir = './work_dirs/train_small_CLAHE'

# 1epoch 대략 43000
cfg.runner.max_iters = 10833*50
cfg.log_config.interval = 3610
cfg.evaluation.interval = 10833
cfg.checkpoint_config.interval = 10833

# Set seed to facitate reproducing the result
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

print(f'Config:\n{cfg.pretty_text}')

Config:
norm_cfg = dict(type='LN', requires_grad=True)
backbone_norm_cfg = dict(type='LN', requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained='pretrain/swin_small_patch4_window7_224.pth',
    backbone=dict(
        type='SwinTransformer',
        pretrain_img_size=224,
        embed_dims=96,
        patch_size=4,
        window_size=7,
        mlp_ratio=4,
        depths=[2, 2, 18, 2],
        num_heads=[3, 6, 12, 24],
        strides=(4, 2, 2, 2),
        out_indices=(0, 1, 2, 3),
        qkv_bias=True,
        qk_scale=None,
        patch_norm=True,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.3,
        use_abs_pos_embed=False,
        act_cfg=dict(type='GELU'),
        norm_cfg=dict(type='LN', requires_grad=True)),
    decode_head=dict(
        type='UPerHead',
        in_channels=[96, 192, 384, 768],
        in_index=[0, 1, 2, 3],
        pool_scales=(1, 2, 3, 6),
        channels=512,
        dropout_ratio=0.1,
        num_c

In [ ]:
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import train_segmentor


# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector

# config_file = 'mmsegmentation/configs/swin/upernet_swin_base_patch4_window12_512x512_160k_ade20k_pretrain_384x384_22K.py'
# model = init_segmentor(config_file, checkpoint_file, device='cuda:0')
model = build_segmentor(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_segmentor(model, datasets, cfg, distributed=False, validate=True, 
                meta=dict())

/opt/conda/lib/python3.7/site-packages/mmcv/utils/misc.py:334: UserWarning: "flip_ratio" is deprecated in `RandomFlip.__init__`, please use "prob" instead
  f'"{src_arg_name}" is deprecated in '
2021-11-02 00:53:21,753 - mmseg - INFO - Loaded 26437 images
/opt/conda/lib/python3.7/site-packages/mmseg/models/backbones/swin.py:553: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '
2021-11-02 00:53:24,376 - mmseg - INFO - Loaded 1508 images
2021-11-02 00:53:24,376 - mmseg - INFO - load checkpoint from pretrained/upernet_swin_small_patch4_window7_512x512_160k_ade20k_pretrain_224x224_1K.pth
2021-11-02 00:53:24,377 - mmseg - INFO - Use load_from_local loader
2021-11-02 00:53:24,593 - mmseg - WARNING - The model and loaded state dict do not match exactly

size mismatch for decode_head.conv_seg.weight: copying a param with shape torch.Size([150, 512, 1, 1]) from checkpoint, the shape in curre

In [ ]:
model